In [1]:
import torch
from time import time
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from zipfile import ZipFile
import numpy as np
import pandas as pd
from time import time
import zipfile
import re
import os
import torchvision
from torchvision import datasets
from torchvision import transforms
import matplotlib.pyplot as plt
from PIL import Image
from torchvision.io import read_image
from matplotlib import image

In [2]:
from vgg_pretrained import vgg_all_freezed,total_trainable_parameters,full_vgg,vgg_layer_freeze
from prepare_dataset import MyDataset,subset_generator
from vgg_pretrained import vgg_all_freezed,total_trainable_parameters,full_vgg,vgg_layer_freeze
from train_fn import train,validation,image_validation,validation_classes,train_batch
from Model import Network,new_model,new_Regressor

In [3]:
fullvgg=full_vgg()

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
transform = transforms.Compose([transforms.ToTensor(),normalize])
target_transform=transforms.Compose([transforms.ToTensor()])


This dataset is done

This dataset is done



In [ ]:
#will take lot of time due to opening each file and creating the dataframe
anno_dir=r"C:\Users\Engineer Protobetz\Documents\Programs Directory\data\AffectNet\train_set\annotations"
image_dir=r"C:\Users\Engineer Protobetz\Documents\Programs Directory\data\AffectNet\train_set\images"
train_data=MyDataset(anno_dir,image_dir,transform=transform, target_transform=None)


In [ ]:

val_anno_dir=r"C:\Users\Engineer Protobetz\Documents\Programs Directory\data\AffectNet\val_set\annotations"
val_image_dir=r"C:\Users\Engineer Protobetz\Documents\Programs Directory\data\AffectNet\val_set\images"
val_data=MyDataset(val_anno_dir,val_image_dir,transform=transform, target_transform=None)

In [ ]:
batch = 64
train_loader=torch.utils.data.DataLoader(train_data,
                                         batch_size=batch,
                                         shuffle=True,
                                         num_workers=4,
                                         pin_memory=True,
                                         sampler=None)
val_loader=torch.utils.data.DataLoader(val_data,
                                       batch_size=batch,
                                       shuffle=False,
                                       num_workers=4,
                                       pin_memory=True,
                                      sampler=None)
val_class_loader=torch.utils.data.DataLoader(val_data,
                                       batch_size=1,
                                       shuffle=False,
                                       num_workers=4,
                                       pin_memory=True,
                                      sampler=None)

In [6]:
df = train_data.basis_to_exp()
X = np.array(df.loc[:, ~df.columns.isin(['class'])])
y = np.array(df['class'])
print(X)
print(y)

[[-0.571429    0.492063    0.75409356 -0.85988782]
 [ 0.295199   -0.120983    0.31902874  1.95975264]
 [ 0.00788395  0.          0.00788395  1.57079633]
 [ 0.463548    0.163295    0.49146923  1.23209885]
 [ 0.561363   -0.00483933  0.56138386  1.57941679]
 [ 0.84127     0.103175    0.84757318  1.44876377]
 [ 0.504202    0.140056    0.52329279  1.29984968]
 [-0.34127     0.722222    0.79879273 -0.44142934]
 [-0.84127     0.468254    0.96280685 -1.06289723]
 [ 0.849206    0.142857    0.86113817  1.4041326 ]
 [ 0.740418   -0.212931    0.77042743  1.85082203]
 [-0.624182   -0.367462    0.7243145  -2.10287277]
 [ 0.126984    0.857143    0.86649816  0.14707819]
 [ 0.329075    0.687185    0.76191442  0.44660439]
 [-0.116144    0.754936    0.76381791 -0.15264933]
 [ 0.666667   -0.539683    0.85773109  2.25131772]
 [ 0.802277    0.120773    0.81131652  1.42138047]
 [ 0.484127    0.142857    0.50476438  1.28385785]
 [ 0.856562    0.256485    0.89413814  1.27985758]
 [-0.0932301  -0.27969     0.29

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
feature_extractor=vgg_layer_freeze(17)
total_trainable_parameters(feature_extractor)

In [ ]:

#input_features_for_denselayer=get_output_features_of_model(model=feature_extractor,batch_size=32,dataloader=train_loader)
input_features_for_denselayer=25088


In [ ]:
modelReg=new_Regressor(feature_extractor,input_features_for_denselayer).to(device)
total_trainable_parameters(modelReg)

In [ ]:
loss_plain=nn.L1Loss()
optimizer=optim.NAdam(modelReg.parameters(),lr=0.0001)
classes = ['Valance', 'Arousal']
classes_exp=['Neutral','Happy','Sad','Surprise','Fear','Disgust','Anger','Contempt']

In [ ]:
epochs = 5
for epoch in range(epochs):
    print(f'Epoch-{epoch}')
    train_batch(train_loader,modelReg,loss_plain,optimizer)
    validation(val_loader,modelReg,loss_plain)

In [ ]:
from tqdm import tqdm
def train12(dataloader,model,loss_fn,optimizer):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.train()
    for batch,(X,y_exp,y_val,y_aro) in enumerate(tqdm(dataloader)):
        y_val = y_val.type(torch.LongTensor).view(-1,1)
        y_aro = y_aro.type(torch.LongTensor).view(-1,1)
        X,y_val,y_aro=X.to(device),y_val.to(device),y_aro.to(device)

        predVal, predAro = model(X)
        print(predVal.shape)
        print(predAro.shape)
        print(y_val.shape)
        print(y_aro.shape)
        break
        """loss1=loss_fn(pred,predVal)
        loss2=loss_fn(pred,predAro)

        optimizer.zero_grad()
        loss1.backward()
        loss2.backward()
        optimizer.step()
        """
train12(train_loader,modelReg,loss_plain,optimizer)

In [10]:
"""
Trains the val and arousal to give the emotion class
The depth is limited to 6 to prevent overfitting and on recognizing complexity from human perspective
The training is independant of the NN but will take the output of the NN to get the emotion class
The dataset X, y will take a lot of time due to the 
"""

from xgboost import XGBRFClassifier

# using random forest due to visible relation
clf = XGBRFClassifier(objective='multi:softmax')

n_estimators = [10,100, 200]
learning_rate = [0.1, 1]
booster = ['dart']
tree_method = ['hist']
eval_metric = ['auc']

gamma = [0.1, 1, 5]
#l1
reg_alpha = [0.1,1,10]
#l2 lambda required
reg_lambda = [0.1,1,10]
n_jobs = [-1]
subsample = [0.33]
max_depth = [6]


grid_state = {'n_estimators':n_estimators,
              'learning_rate':learning_rate,
              'booster':booster , 
              'tree_method':tree_method,
              'n_jobs':n_jobs,
              'eval_metric':eval_metric,
              'reg_alpha':reg_alpha,
              'reg_lambda':reg_lambda,
              'gamma': gamma,
              'subsample':subsample,
              'max_depth':max_depth
             }

xgb_grid = GridSearchCV(estimator = clf,
                        param_grid = grid_state,
                        scoring = "roc_auc",
                        cv=5,
                        verbose=0,
                    )
xgb_grid.fit(X,y)

[20:29:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
{'colsample_bynode': 0.8, 'learning_rate': 1.0, 'reg_lambda': 1e-05, 'subsample': 0.8, 'use_label_encoder': True, 'objective': 'multi:softprob', 'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bytree': 1, 'enable_categorical': False, 'gamma': 0, 'gpu_id': -1, 'importance_type': None, 'interaction_constraints': '', 'max_delta_step': 0, 'max_depth': 6, 'min_child_weight': 1, 'missing': nan, 'monotone_constraints': '()', 'n_estimators': 100, 'n_jobs': 16, 'num_parallel_tree': 100, 'predictor': 'auto', 'random_state': 0, 'reg_alpha': 0, 'scale_pos_weight': None, 'tree_method': 'exact', 'validate_parameters': 1, 'verbosity': None}


In [ ]:
def predict(clf, reg):
    # to write once someone has the NN model
    